## Ejercicio práctico LLMs

Como parte de un proyecto en el que estás creando un recomendador de películas en español, necesitamos desarrollar un `pipeline` que **extraiga
información estructurada de descripciones de películas** que hemos scrapeamos de la web.

El objetivo del ejercicio es usar uno de los LLMs vistos en clase (el `Qwen/Qwen2.5-0.5B-Instruct`) para procesar estas descripciones y extraer los campos
`Título`, `Director` y `Año estreno`.

Instrucciones adicionales:
- Se pueden reutilizar funciones y código del *notebook* que hemos visto durante la clase.
- El alumno tiene libertad para usar cualquier técnica de prompting, parámetros, pre-procesado del texto, postprocesado de la respuesta, etc. No hace falta ceñirse sólo
a las técnicas y parámetros vistos en clase.
- Puede haber más de una iteración con el modelo antes de obtener la respuesta final, se creativo!
- Dos cosas prohibidas:
  - Incluir los textos proporcionados como `test` en el propio *prompt*. Imaginemos que esos textos de test son los datos que recibimos cuando el modeloe stá en producción, es decir, no es información de la que disponemos durante el desarrollo.
  - Usar otro modelo para generar texto que no sea el `Qwen/Qwen2.5-0.5B-Instruct` 
- Si un campo no está presente en el texto, se debe devolver un string vacío `""`.
- Cuidado porque algunos textos puden incluir información confusa! Por ejemplo, pueden incluir titulos de otras pelis, nombres de personas que no son el director/directoa (ej. actores), etc.
  
Asegúrate de que el notebook que subes a la plataforma cumpla que:
* Se puede ejecutar sin fallos (es decir, si le doy a "Run all" se ejecutará sin problema)
* Ha sido ejecutado previamente, de modo que pueda ver los resultados sin necesidad de ejecutarlo

In [7]:
# NO EDITAR ESTA CELDA
SMALL_CHAT_MODEL_CON_CASTELLANO =  "Qwen/Qwen2.5-0.5B-Instruct"
DEVICE = "cuda:0" # Si no tenéis GPU cambiar a "cpu"
# Si usáis un Google Colab ó un entorno con GPU podéis dejarlo en "cuda:0"

from transformers import AutoModelForCausalLM, AutoTokenizer
# Veamos cuánta memoria usa el modelo
if DEVICE == "cuda:0":
    import torch
    mem0 = torch.cuda.memory_allocated(DEVICE)
else:
    # Memoria RAM usada actualmnente en el sistema
    import os
    import psutil
    mem0 = psutil.virtual_memory().used

model = AutoModelForCausalLM.from_pretrained(SMALL_CHAT_MODEL_CON_CASTELLANO).to(DEVICE)
tokenizer = AutoTokenizer.from_pretrained(SMALL_CHAT_MODEL_CON_CASTELLANO)

if DEVICE == "cuda:0":
    mem1 = torch.cuda.memory_allocated(DEVICE)
else:
    mem1 = psutil.virtual_memory().used

print(f"Modelo cargado! Memoria usada: ~{(mem1-mem0)/(1024**3):.1f} GB")

Modelo cargado! Memoria usada: ~0.0 GB


In [8]:
def pipeline_completo(descripcion_peli: str, model, tokenizer) -> dict[str, dict[str, str]]:

    respuesta = {}

    ## Pre-procesar el texto si es necesario

    ## Crear prompt

    ## Tokenizar el prompt

    ## Generar respuesta con `model`

    ## Postprocesar respuesta y convertirla en diccionario

    ## Retornar respuesta
    return respuesta

In [9]:
# NO EDITAR ESTA CELDA
from typing import Callable
from tqdm import tqdm
def evaluar_pipeline_con_conjunto_de_test(ejemplos: list[dict[str, dict]], tu_pipeline: Callable, **pipeline_kwargs) -> float:
    """
    Obtener el ratio de acierto (del 0% al 100%) sobre los ejemplos de test.
    """
    total_campos, total_aciertos = 0, 0
    for i, ejemplo in enumerate(ejemplos):
        texto, resp_correcta = ejemplo["texto"], ejemplo["respuesta"]
        resp = tu_pipeline(texto, **pipeline_kwargs)
        total_campos += 3
        if isinstance(resp, dict): # La salida de tu pipeline debería ser un diccionario
            print(f"Evaluando {i + 1}/{len(ejemplos)}..")
            if "Título" in resp:
                acierto = int(resp["Título"].strip().lower() == resp_correcta["Título"].strip().lower())
                if acierto:
                    print(f"Título correcto!")
                total_aciertos += acierto
            if "Director" in resp:
                acierto = int(resp["Director"].strip().lower() == resp_correcta["Director"].strip().lower())
                if acierto :
                    print(f"Director correcto!")
                total_aciertos += acierto
            if "Año estreno" in resp:
                acierto = int(str(resp["Año estreno"]) == str(resp_correcta["Año estreno"]))
                if acierto:
                    print(f"Año de estreno correcto!")
                total_aciertos += acierto

        print(f"Tasa de acierto acumulada: {total_aciertos/total_campos:.1%}")

    tasa_aciertos = total_aciertos / total_campos
    print(f"El ratio de aciertos es del {tasa_aciertos*100:.1f}%")

    return tasa_aciertos

TEXTOS_TEST = [
    {
      "texto": "En *El Resplandor*, dirigida por Stanley Kubrick, un escritor se convierte en el cuidador de invierno de un aislado hotel de montaña, donde descubre horrores sobrenaturales que afectan a su familia. La película se estrenó en 1980.",
      "respuesta": {
        "Título": "El Resplandor",
        "Director": "Stanley Kubrick",
        "Año estreno": "1980"
      }
    },
    {
      "texto": "La película Titanic, lanzada en 1997 y dirigida por James Cameron, narra la trágica historia de amor entre Jack y Rose, dos pasajeros del famoso transatlántico que se hundió en su viaje inaugural.",
      "respuesta": {
        "Título": "Titanic",
        "Director": "James Cameron",
        "Año estreno": "1997"
      }
    },
    {
      "texto": "'Coco' es una animación de Disney Pixar que se sumerge en la cultura mexicana y sigue a un joven llamado Miguel, que sueña con convertirse en músico a pesar de la prohibición de su familia. La película fue estrenada en 2017.",
      "respuesta": {
        "Título": "Coco",
        "Director": "",
        "Año estreno": "2017"
      }
    },
    {
      "texto": "En 1994, El Rey León se convirtió en un clásico instantáneo de Disney, narrando la historia de Simba, un joven león que debe encontrar su lugar como rey de la sabana africana.",
      "respuesta": {
        "Título": "El Rey León",
        "Director": "",
        "Año estreno": "1994"
      }
    },
    {
      "texto": "*Pulp Fiction*, dirigida por Quentin Tarantino, es un entrelazado de historias sobre crimen y violencia en Los Ángeles. La película, estrenada en 1994, es conocida por su diálogo ingenioso y su estructura no lineal.",
      "respuesta": {
        "Título": "Pulp Fiction",
        "Director": "Quentin Tarantino",
        "Año estreno": "1994"
      }
    },
    {
      "texto": "La película Parásitos, ganadora del Óscar, sumerge al espectador en la vida de dos familias coreanas de extremos sociales opuestos que se entrelazan de manera inesperada. Su director, Bong Joon-ho, captura magistralmente la lucha de clases.",
      "respuesta": {
        "Título": "Parásitos",
        "Director": "Bong Joon-ho",
        "Año estreno": ""
      }
    },
    {
      "texto": "*La La Land*, un musical romántico lanzado en 2016, sigue la historia de una aspirante a actriz y un músico de jazz que luchan por hacer realidad sus sueños en Los Ángeles. La película fue dirigida por Damien Chazelle.",
      "respuesta": {
        "Título": "La La Land",
        "Director": "Damien Chazelle",
        "Año estreno": "2016"
      }
    },
    {
      "texto": "Un hito del cine mudo, Metrópolis, es una obra maestra de la ciencia ficción dirigida por Fritz Lang. Estrenada en 1927, la película presenta una futurística ciudad dividida entre trabajadores y la élite gobernante.",
      "respuesta": {
        "Título": "Metrópolis",
        "Director": "Fritz Lang",
        "Año estreno": "1927"
      }
    },
    {
      "texto": "*Interstellar*, una odisea espacial que explora los límites del amor y el sacrificio humano a través de un viaje interestelar en busca de un nuevo hogar para la humanidad, fue dirigida por Christopher Nolan y se estrenó en 2014.",
      "respuesta": {
        "Título": "Interstellar",
        "Director": "Christopher Nolan",
        "Año estreno": "2014"
      }
    },
    {
      "texto": "Mujercitas, de 2019, es una adaptación del clásico de Louisa May Alcott, diriga por Greta Gerwig, que sigue a las hermanas March en su paso de la infancia a la adultez",
      "respuesta": {
        "Título": "Mujercitas",
        "Director": "Greta Gerwig",
        "Año estreno": "2019"
      }
    },
    {
        "texto": "En 'Gravity' (2013), Sandra Bullock y George Clooney protagonizan como astronautas que luchan por sobrevivir después de que su transbordador espacial se destruye, dejándolos a la deriva en el espacio. Esta tensa y visualmente impresionante película explora la vastedad y el peligro del espacio exterior",
        "respuesta": {
            "Título": "Gravity",
            "Director": "",
            "Año estreno": "2013"
        }
    },
    {
        "texto": "El lobo de Wall Street narra la historia real de Jordan Belfort, interpretado por Leonardo DiCaprio, un corredor de bolsa de Nueva York que se enriquece a través de la corrupción y el fraude. La película, que también cuenta con la actuación de Jonah Hill y Margot Robbie, sumerge al espectador en el extravagante estilo de vida de Belfort y su eventual caída. Sin mencionar al director, la película captura el frenesí financiero de los años 90",
        "respuesta": {
            "Título": "El lobo de Wall Street",
            "Director": "",
            "Año estreno": ""
        }
    },
    {
        "texto": "Dirigida por Ridley Scott, *Blade Runner* es un clásico de ciencia ficción ambientado en el año 2019, donde Harrison Ford interpreta a Rick Deckard, un 'blade runner' encargado de cazar replicantes rebeldes. A pesar de su ambientación futurista en 2019, la película fue estrenada en 1982, convirtiéndose en una obra de culto por su profunda reflexión sobre la humanidad, la tecnología y el alma.",
        "respuesta": {
            "Título": "Blade Runner",
            "Director": "Ridley Scott",
            "Año estreno": "1982"
        }
    },
     {
    "texto": "\"Los Increíbles\", de Pixar, lanzada en 2004, es una animada aventura sobre una familia de superhéroes que intenta vivir una vida suburbana normal hasta que se ven obligados a volver a la acción.",
    "respuesta": {
      "Título": "Los Increíbles",
      "Director": "",
      "Año estreno": "2004"
    }
  },
  {
    "texto": "\"Forrest Gump\", protagonizada por Tom Hanks, es una emotiva historia de la vida de un hombre sencillo con un corazón puro y su extraordinaria jornada a través de eventos históricos de Estados Unidos en la segunda mitad del siglo XX. Estrenada en 1994.",
    "respuesta": {
      "Título": "Forrest Gump",
      "Director": "",
      "Año estreno": "1994"
    }
  },
  {
    "texto": "En \"El Club de la Pelea\", dirigida por David Fincher y lanzada en 1999, un empleado insomne y un vendedor de jabón carismático forman un club de lucha subterráneo que evoluciona en algo mucho más grande.",
    "respuesta": {
      "Título": "El Club de la Pelea",
      "Director": "David Fincher",
      "Año estreno": "1999"
    }
  },
  {
    "texto": "La comedia de 2014 \"El Gran Hotel Budapest\", dirigida por Wes Anderson, narra la historia de un conserje legendario en un famoso hotel europeo entre las guerras y su amistad con un joven empleado.",
    "respuesta": {
      "Título": "El Gran Hotel Budapest",
      "Director": "Wes Anderson",
      "Año estreno": "2014"
    }
  },
  {
    "texto": "\"Mad Max: Furia en la Carretera\", dirigida por George Miller en 2015, revive la franquicia con una persecución post-apocalíptica intensa y visualmente deslumbrante a través del desierto.",
    "respuesta": {
      "Título": "Mad Max: Furia en la Carretera",
      "Director": "George Miller",
      "Año estreno": "2015"
    }
  },
    {
    "texto": "\"El Silencio de los Inocentes\", lanzada en 1991, es un inquietante thriller psicológico donde una joven agente del FBI busca la ayuda de Hannibal Lecter, un brillante psiquiatra y asesino en serie encarcelado, para capturar a otro asesino.",
    "respuesta": {
      "Título": "El Silencio de los Inocentes",
      "Director": "",
      "Año estreno": "1991"
    }
  },
  {
    "texto": "El largometraje Gladiador, protagonizado por Russell Crowe y dirigida por Ridley Scott en 2000, presenta la épica historia de un general romano traicionado que se convierte en gladiador para vengar la muerte de su familia.",
    "respuesta": {
      "Título": "Gladiador",
      "Director": "Ridley Scott",
      "Año estreno": "2000"
    }
  },
  {
    "texto": "\"Amélie\", una película francesa de 2001, sigue a una joven inocente y soñadora en París que decide cambiar la vida de las personas a su alrededor para mejor mientras lucha con su propia soledad.",
    "respuesta": {
      "Título": "Amélie",
      "Director": "",
      "Año estreno": "2001"
    }
  },
  {
    "texto": "\"Jurassic Park\", dirigida por Steven Spielberg en 1993, revolucionó el cine con sus efectos especiales al traer dinosaurios a la vida en esta aventura emocionante en un parque temático de isla.",
    "respuesta": {
      "Título": "Jurassic Park",
      "Director": "Steven Spielberg",
      "Año estreno": "1993"
    }
  },
  {
    "texto": "\"Inception\" (Origen), dirigida por Christopher Nolan en 2010, es un thriller de ciencia ficción que explora el robo de secretos a través del subconsciente durante el sueño, llevando a cabo una idea compleja: la inserción de otra idea.",
    "respuesta": {
      "Título": "Inception",
      "Director": "Christopher Nolan",
      "Año estreno": "2010"
    }
  },
  {
    "texto": "La película italiana 'La Vida es Bella', de 1997, cuenta la historia de un padre judío que usa su imaginación para proteger a su hijo de los horrores de un campo de concentración nazi.",
    "respuesta": {
      "Título": "La Vida es Bella",
      "Director": "",
      "Año estreno": "1997"
    }
  }
]

In [11]:
evaluar_pipeline_con_conjunto_de_test(TEXTOS_TEST, pipeline_completo, model=model, tokenizer=tokenizer)

Evaluando 1/24..
Tasa de acierto acumulada: 0.0%
Evaluando 2/24..
Tasa de acierto acumulada: 0.0%
Evaluando 3/24..
Tasa de acierto acumulada: 0.0%
Evaluando 4/24..
Tasa de acierto acumulada: 0.0%
Evaluando 5/24..
Tasa de acierto acumulada: 0.0%
Evaluando 6/24..
Tasa de acierto acumulada: 0.0%
Evaluando 7/24..
Tasa de acierto acumulada: 0.0%
Evaluando 8/24..
Tasa de acierto acumulada: 0.0%
Evaluando 9/24..
Tasa de acierto acumulada: 0.0%
Evaluando 10/24..
Tasa de acierto acumulada: 0.0%
Evaluando 11/24..
Tasa de acierto acumulada: 0.0%
Evaluando 12/24..
Tasa de acierto acumulada: 0.0%
Evaluando 13/24..
Tasa de acierto acumulada: 0.0%
Evaluando 14/24..
Tasa de acierto acumulada: 0.0%
Evaluando 15/24..
Tasa de acierto acumulada: 0.0%
Evaluando 16/24..
Tasa de acierto acumulada: 0.0%
Evaluando 17/24..
Tasa de acierto acumulada: 0.0%
Evaluando 18/24..
Tasa de acierto acumulada: 0.0%
Evaluando 19/24..
Tasa de acierto acumulada: 0.0%
Evaluando 20/24..
Tasa de acierto acumulada: 0.0%
Evaluando

0.0